# Import Module

In [35]:
from functools import partial
import os
import pickle

from hyperopt import hp
import numpy as np
import pandas as pd
from ray import tune
from ray.tune.suggest.hyperopt import HyperOptSearch
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
import torch
from torch import nn
from torch import optim
from torch.utils.data import TensorDataset, DataLoader

from models import AttentionRNN
from train_model import train_timeseries_net

In [36]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load Data

In [37]:
x = pd.read_csv("./data/2_X_train.csv").drop(columns="Temp").values
y = pd.read_csv("./data/2_Y_train.csv").values.reshape(-1)

sequence_length = 16
num_days = int(x.shape[0] / sequence_length)
feature_size = x.shape[1]
continuous_feature_size = 8

x = x.reshape(num_days, sequence_length, feature_size)
y = y.reshape(num_days, sequence_length, 1)

# Utils

In [38]:
def pipeline_rnn(train_x, train_y, val_x, val_y, test_x, test_y, options, config, trainer=train_timeseries_net):
    # preprocessing
    train_loader, val_x, val_y, test_x, test_y = preprocess_for_rnn(train_x, train_y, val_x, val_y, test_x, test_y)

    # training and params optimization
    options["dataset"]["train_loader"] = train_loader
    options["dataset"]["val_x"] = val_x
    options["dataset"]["val_y"] = val_y

    hyperopt = HyperOptSearch(metric="loss", mode="min")

    analysis = tune.run(
        partial(trainer, options=options),
        config=config,
        num_samples=2,
        search_alg=hyperopt,
        resources_per_trial={'gpu':1}
    )

    # test
    model = standby_rnn_for_test(analysis, options)
    acc = test(model, test_x, test_y)
    return acc

In [39]:
def preprocess_for_rnn(train_x, train_y, val_x, val_y, test_x, test_y):

    train_num_days = train_x.shape[0]
    val_num_days = val_x.shape[0]
    test_num_days = test_x.shape[0]

    train_x = train_x.reshape(train_num_days * sequence_length, feature_size)
    val_x = val_x.reshape(val_num_days * sequence_length, feature_size)
    test_x = test_x.reshape(test_num_days * sequence_length, feature_size)

    ss = preprocessing.StandardScaler()
    ss.fit(train_x[:, :continuous_feature_size])

    train_x[:, :continuous_feature_size] = \
        ss.transform(train_x[:, :continuous_feature_size])

    val_x[:, :continuous_feature_size] = \
        ss.transform(val_x[:, :continuous_feature_size])

    test_x[:, :continuous_feature_size] = \
        ss.transform(test_x[:, :continuous_feature_size])

    train_x = train_x.reshape(train_num_days, sequence_length, feature_size)
    val_x = val_x.reshape(val_num_days, sequence_length, feature_size)
    test_x = test_x.reshape(test_num_days, sequence_length, feature_size)

    train_x = torch.tensor(train_x, dtype=torch.float32)
    train_y = torch.tensor(train_y, dtype=torch.float32)
    test_x = torch.tensor(test_x, dtype=torch.float32)
    test_y = torch.tensor(test_y, dtype=torch.float32)
    val_x = torch.tensor(val_x, dtype=torch.float32)
    val_y = torch.tensor(val_y, dtype=torch.float32)

    train_x = train_x.to(device)
    train_y = train_y.to(device)
    test_x = test_x.to(device)
    test_y = test_y.to(device)
    val_x = val_x.to(device)
    val_y = val_y.to(device)

    train_ds = TensorDataset(train_x, train_y)
    train_loader = DataLoader(train_ds, batch_size=6, shuffle=True)
    
    return train_loader, val_x, val_y, test_x, test_y

In [40]:
def standby_rnn_for_test(analysis, options):
    best = analysis.get_best_config(metric="loss", mode="min")
    logdir = analysis.get_best_logdir("loss", mode="min")
    state_dict = torch.load(os.path.join(logdir, "rnn.pth"))

    model = AttentionRNN(input_size=options["params"]["input_size"], hidden_size=best["hidden_size"],
                         num_layers=options["params"]["num_layers"], num_classes=options["params"]["num_classes"],
                         fc_sizes=[best["fc_size_0"], best["fc_size_1"], best["fc_size_2"]]).to(device)
    model.load_state_dict(state_dict)
    return model

In [41]:
def test(model, test_x, test_y):
    model.eval()
    score_y = model(test_x, device).reshape(-1)
    score_y = torch.sigmoid(score_y)
    pred_y = torch.tensor([1 if i > 0.5 else 0 for i in score_y]).to(device)
    test_y = test_y.reshape(-1)
    test_y = test_y.to('cpu').detach().numpy().copy()
    pred_y = pred_y.to('cpu').detach().numpy().copy()
    acc = sum(pred_y == test_y) / int(test_y.shape[0])
    return acc

In [42]:
def pipeline_benchmarks(train_x,train_y,test_x,test_y,val_x,val_y, config, options, trainer, name):
    # preprocessing
    train_x = train_x.reshape(-1, train_x.shape[2])
    train_y = train_y.reshape(-1)
    test_x = test_x.reshape(-1, test_x.shape[2])
    test_y = test_y.reshape(-1)
    val_x = val_x.reshape(-1, val_x.shape[2])
    val_y = val_y.reshape(-1)
    options["dataset"]["train_x"] = train_x
    options["dataset"]["train_y"] = train_y
    options["dataset"]["val_x"] = val_x
    options["dataset"]["val_y"] = val_y
    
    # training and params optimization
    hyperopt = HyperOptSearch(metric="accuracy", mode="max")
    analysis = tune.run(
        partial(trainer, options=options),
        config=config,
        num_samples=2,
        search_alg=hyperopt,
        resources_per_trial={'gpu':1}
    )   
    
    # test
    model = standby_benchmarks_for_test(analysis, name)
    pred_y = model.predict(test_x)
    acc = sum(pred_y == test_y) / int(test_y.shape[0])
    return acc

In [43]:
def train_rf(config, options):
    train_x, train_y, val_x, val_y = options["dataset"].values()
    rf = RandomForestClassifier(max_depth=config["max_depth"],min_samples_leaf=config["min_samples_leaf"],
                                min_samples_split=config["min_samples_split"], n_estimators=config["n_estimators"])
    rf.fit(train_x, train_y)
    
    pred_y = rf.predict(val_x)
    acc = sum(val_y == pred_y) / pred_y.shape[0]
    tune.report(accuracy=acc)
    
    with open(f"./rf.pkl", "wb") as out_file:
        pickle.dump(rf, out_file)

In [44]:
def train_linear(config, options):
    train_x, train_y, val_x, val_y = options["dataset"].values()
    linear = make_pipeline(preprocessing.StandardScaler(), SVC(kernel="linear", C = config["C"]))
    linear.fit(train_x, train_y)
    
    pred_y = linear.predict(val_x)
    acc = sum(val_y == pred_y) / pred_y.shape[0]
    tune.report(accuracy=acc)
    
    with open(f"./linear.pkl", "wb") as out_file:
        pickle.dump(linear, out_file)

In [45]:
def train_rbf(config, options):
    train_x, train_y, val_x, val_y = options["dataset"].values()
    linear = make_pipeline(preprocessing.StandardScaler(), SVC(kernel="rbf", C = config["C"], gamma=config["gamma"]))
    linear.fit(train_x, train_y)
    
    pred_y = linear.predict(val_x)
    acc = sum(val_y == pred_y) / pred_y.shape[0]
    tune.report(accuracy=acc)
    
    with open(f"./rbf.pkl", "wb") as out_file:
        pickle.dump(linear, out_file)

In [46]:
def standby_benchmarks_for_test(analysis, name):
    best = analysis.get_best_config(metric="accuracy", mode="min")
    logdir = analysis.get_best_logdir("accuracy", mode="min")
    with open(os.path.join(logdir, name), 'rb') as f:
        model = pickle.load(f)
    return model

# Set Config and Options for Raytune

In [47]:
rnn_config = {
    "lr": tune.uniform(0, 0.1),
    "hidden_size": tune.choice([25, 50, 100, 150]),
    "weight_decay": tune.choice([0, 1e-3, 1e-1]),
    "eps": tune.choice([1e-11, 1e-8, 1e-5]),
    "fc_size_0": tune.choice([50, 75, 100]),
    "fc_size_1": tune.choice([15, 25, 35]),
    "fc_size_2": tune.choice([5, 10, 20]),
}

In [48]:
rf_config = {
    'max_depth': tune.choice([5, 10, 15, 20, 50, None]),
    'min_samples_leaf': tune.choice([3, 4, 5]),
    'min_samples_split': tune.choice([2, 8, 10, 12]),
    'n_estimators': tune.choice([50, 100, 200])
}

linear_config = {
    'C': tune.choice([0.1, 0.5, 1, 5, 10, 50, 100]),
}

rbf_config = {
    'C': tune.choice([0.1, 0.5, 1, 5, 10, 50, 100]),
    "gamma": tune.loguniform(0.0001, 1)
}

In [49]:
rnn_options = {
    'dataset': {'train_loader': None, 'val_x': None, 'val_y': None},
    "num_epochs": 50,
    "params": {"input_size": 13, "num_layers": 1, "num_classes": 1},
    "device": device
}

benchmarks_options = {
    'dataset': {'train_x': None, 'train_y': None,
                'val_x': None, 'val_y': None},
}

# Raytune

In [50]:
n_splits = 2
val_size = 0.1
num_repeats = 1

kf = KFold(n_splits=n_splits, random_state=1, shuffle=True)

running_acc = 0
rf_running_acc = 0
linear_running_acc = 0
rbf_running_acc = 0

accs = []
rf_accs = []
linear_accs = []
rbf_accs = []

for repeat in range(num_repeats):
    kf = KFold(n_splits=n_splits, random_state=repeat, shuffle=True)
    for train_idx, test_idx in kf.split(x):
        # split data into train, test
        train_x, test_x = x[train_idx], x[test_idx]
        train_y, test_y = y[train_idx], y[test_idx]
        train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=val_size, train_size=(1-val_size))

        # BiLSTM-Attention
        running_acc += pipeline_rnn(
            train_x, train_y, test_x, test_y,
            val_x, val_y, options=rnn_options, config=rnn_config,
            trainer=train_timeseries_net)

        # Benchmarks
        rf_running_acc += pipeline_benchmarks(
            train_x, train_y, test_x, test_y, 
            val_x, val_y, config=rf_config,
            options=benchmarks_options, trainer=train_rf, name="rf.pkl")

        linear_running_acc += pipeline_benchmarks(
            train_x, train_y, test_x, test_y,
            val_x, val_y, config=linear_config,
            options=benchmarks_options, trainer=train_linear, name="linear.pkl")

        rbf_running_acc += pipeline_benchmarks(
            train_x, train_y, test_x, test_y,
            val_x, val_y, config=rbf_config,
            options=benchmarks_options, trainer=train_rbf, name="rbf.pkl")
        
    mean_acc = running_acc/n_splits
    rf_mean_acc = rf_running_acc/n_splits
    linear_mean_acc = linear_running_acc/n_splits
    rbf_mean_acc = rbf_running_acc/n_splits
    
    
    accs.append(mean_acc)
    rf_accs.append(rf_mean_acc)
    linear_accs.append(linear_mean_acc)
    rbf_accs.append(rbf_mean_acc)

2022-05-14 13:42:49,187	INFO trial_runner.py:803 -- starting train_timeseries_net_4e8babfa


Trial name,status,loc,eps,fc_size_0,fc_size_1,fc_size_2,hidden_size,lr,weight_decay
train_timeseries_net_4e8babfa,RUNNING,192.168.177.120:2749,1e-11,75,25,10,150,0.065986,0.1


Result for train_timeseries_net_4e8babfa:
  date: 2022-05-14_13-42-52
  done: false
  experiment_id: 526333250ca34529af643f3380780f51
  hostname: higuchilab-P7xxTM1
  iterations_since_restore: 1
  loss: 0.6843794584274292
  node_ip: 192.168.177.120
  pid: 2749
  time_since_restore: 0.37114930152893066
  time_this_iter_s: 0.37114930152893066
  time_total_s: 0.37114930152893066
  timestamp: 1652503372
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 4e8babfa
  warmup_time: 0.0017306804656982422
  


Trial name,status,loc,eps,fc_size_0,fc_size_1,fc_size_2,hidden_size,lr,weight_decay,iter,total time (s),loss
train_timeseries_net_4e8babfa,RUNNING,192.168.177.120:2749,1e-11,75,25,10,150,0.065986,0.1,5,1.87325,0.678357
train_timeseries_net_508888a6,PENDING,,1e-08,50,35,5,50,0.03276,0.001,,,


Result for train_timeseries_net_4e8babfa:
  date: 2022-05-14_13-42-58
  done: false
  experiment_id: 526333250ca34529af643f3380780f51
  hostname: higuchilab-P7xxTM1
  iterations_since_restore: 15
  loss: 0.6747162938117981
  node_ip: 192.168.177.120
  pid: 2749
  time_since_restore: 5.596039772033691
  time_this_iter_s: 0.3825995922088623
  time_total_s: 5.596039772033691
  timestamp: 1652503378
  timesteps_since_restore: 0
  training_iteration: 15
  trial_id: 4e8babfa
  warmup_time: 0.0017306804656982422
  


...

Result for train_rbf_6ffc8f8e:
  accuracy: 0.9
  date: 2022-05-14_13-43-46
  done: false
  experiment_id: e43573a49d71454f9f1af839c132a060
  hostname: higuchilab-P7xxTM1
  iterations_since_restore: 1
  node_ip: 192.168.177.120
  pid: 4296
  time_since_restore: 0.02266693115234375
  time_this_iter_s: 0.02266693115234375
  time_total_s: 0.02266693115234375
  timestamp: 1652503426
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 6ffc8f8e
  warmup_time: 0.0018582344055175781
  
Result for train_rbf_6ffc8f8e:
  accuracy: 0.9
  date: 2022-05-14_13-43-46
  done: true
  experiment_id: e43573a49d71454f9f1af839c132a060
  experiment_tag: 1_C=0.5,gamma=0.058102
  hostname: higuchilab-P7xxTM1
  iterations_since_restore: 1
  node_ip: 192.168.177.120
  pid: 4296
  time_since_restore: 0.02266693115234375
  time_this_iter_s: 0.02266693115234375
  time_total_s: 0.02266693115234375
  timestamp: 1652503426
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 6ffc8f8e
  warmup_t

2022-05-14 13:43:46,815	INFO trial_runner.py:803 -- starting train_rbf_70c27a46


Result for train_rbf_70c27a46:
  accuracy: 0.8875
  date: 2022-05-14_13-43-48
  done: false
  experiment_id: 289deb886df84130a961e1f7cd57104b
  hostname: higuchilab-P7xxTM1
  iterations_since_restore: 1
  node_ip: 192.168.177.120
  pid: 4335
  time_since_restore: 0.02065420150756836
  time_this_iter_s: 0.02065420150756836
  time_total_s: 0.02065420150756836
  timestamp: 1652503428
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 70c27a46
  warmup_time: 0.0016965866088867188
  
Result for train_rbf_70c27a46:
  accuracy: 0.8875
  date: 2022-05-14_13-43-48
  done: true
  experiment_id: 289deb886df84130a961e1f7cd57104b
  experiment_tag: 2_C=1,gamma=0.016106
  hostname: higuchilab-P7xxTM1
  iterations_since_restore: 1
  node_ip: 192.168.177.120
  pid: 4335
  time_since_restore: 0.02065420150756836
  time_this_iter_s: 0.02065420150756836
  time_total_s: 0.02065420150756836
  timestamp: 1652503428
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 70c27a46
  warm

Trial name,status,loc,C,gamma,iter,total time (s),accuracy
train_rbf_6ffc8f8e,TERMINATED,192.168.177.120:4296,0.5,0.0581017,1,0.0226669,0.9
train_rbf_70c27a46,TERMINATED,192.168.177.120:4335,1,0.0161059,1,0.0206542,0.8875


2022-05-14 13:43:48,127	INFO tune.py:701 -- Total run time: 2.95 seconds (2.84 seconds for the tuning loop).


In [52]:
print(f"BiLSTM-Attention:{np.mean(accs)}({np.std(accs)})")
print(f"RF:{np.mean(rf_accs)}({np.std(accs)})")
print(f"Linear-SVM:{np.mean(linear_accs)}({np.std(accs)})")
print(f"Rbf-SVM:{np.mean(rbf_accs)}({np.std(accs)})")

BiLSTM-Attention:0.88125(0.0)
RF:0.8960459183673469(0.0)
Linear-SVM:0.8654336734693877(0.0)
Rbf-SVM:0.8724489795918366(0.0)
